In [1]:
from data_collection.core.currency import CurrencyPair
from typing import *

import requests
import re

<h4>Collect all tickers available on Binance</h4>

In [2]:
resp = requests.get("https://api.binance.com/api/v3/exchangeInfo")

data: Dict[str, Any] = resp.json()

In [4]:
currency_pairs: List[CurrencyPair] = [
    CurrencyPair(base=entry["baseAsset"], term=entry["quoteAsset"]) for entry in data["symbols"]
]

<h4>Reverse engineer BinanceDataVision</h4>

In [8]:
params: Dict[str, Any] = {
    "delimiter": "/",
    "prefix": "data/spot/daily/trades/1INCHBUSD/"
}

resp = requests.get(
    "https://s3-ap-northeast-1.amazonaws.com/data.binance.vision",
    params=params
)

In [10]:
hrefs: List[str] = re.findall(pattern=r"<Key>(.*?)</Key>", string=resp.text)
hrefs = [href for href in hrefs if "CHECKSUM" not in href]

len(hrefs)

500

In [11]:
hrefs[1]

'data/spot/daily/trades/1INCHBUSD/1INCHBUSD-trades-2021-02-24.zip'